## Intro Concepts: Adding Memory
### 1. Load the environment variables

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

### 2. Set up the model and imports:

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatOpenAI(model="gpt-3.5-turbo")

### 3. Create a workflow with memory:
We'll create a simple chat application that remembers the conversation history.

Define the graph state to be a list of messages

In [17]:
workflow = StateGraph(state_schema=MessagesState)

Define the function that calls our model

In [18]:
def call_model(state: MessagesState):
    # Get all messages from the state
    response = model.invoke(state["messages"])
    
    # Return AI's response as a message
    return {"messages": response}

Add the model node to the graph

In [19]:
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

Add memory

In [20]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### 4. Test the conversation with memory

In [21]:
config = {"configurable": {"thread_id": "user123"}}

First message

In [22]:
messages = [
    SystemMessage(content="You are a friendly GDGOnCampus AI assistant."),
    HumanMessage(content="Hi! I'm Wahib.")
]

result = app.invoke({"messages": messages}, config)

In [23]:
print("AI Response:")
print(result["messages"][-1].content)

AI Response:
Hello Wahib! How can I assist you today?


Second message - the model should remember the user's name

In [24]:
result = app.invoke(
    {"messages": [HumanMessage("What's my name?")]},
    config
)

print("\nAI Response:")
print(result["messages"][-1].content)


AI Response:
Your name is Wahib.


### 5. Check the conversation history
Get the full conversation history

In [25]:
state = app.get_state(config).values

print("Full Conversation History:")
for message in state["messages"]:
    print(f"\n{message.type}: {message.content}")

Full Conversation History:

system: You are a friendly GDGOnCampus AI assistant.

human: Hi! I'm Wahib.

ai: Hello Wahib! How can I assist you today?

human: What's my name?

ai: Your name is Wahib.
